In [1]:
import sys
sys.path.append('/home/rida/GNN')

In [ ]:
import torch
import numpy as np
from torch_geometric.data import Data
from sklearn.linear_model   import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics        import accuracy_score
import Embedding_phase.Generate_data.config as args
from Embedding_phase.Generate_data.Generate_training import transform_to_float
from Embedding_phase.MGCN.Multi_GCN import MGCN
import json
import glob
import os
from collections import Counter

In [57]:
data = torch.load(args.DATA_PATH_6_2, weights_only=False)

In [ ]:
data = transform_to_float(data)

In [59]:
data

[(Data(x=[3246, 18], edge_index=[2, 30648], edge_type=[30648], edge_score=[30648]),
  Data(x=[5670, 18], edge_index=[2, 93016], edge_type=[93016], edge_score=[93016]),
  1),
 (Data(x=[9174, 18], edge_index=[2, 158386], edge_type=[158386], edge_score=[158386]),
  Data(x=[5670, 18], edge_index=[2, 93016], edge_type=[93016], edge_score=[93016]),
  -1),
 (Data(x=[3265, 18], edge_index=[2, 37040], edge_type=[37040], edge_score=[37040]),
  Data(x=[5185, 18], edge_index=[2, 92648], edge_type=[92648], edge_score=[92648]),
  1),
 (Data(x=[8573, 18], edge_index=[2, 131770], edge_type=[131770], edge_score=[131770]),
  Data(x=[5185, 18], edge_index=[2, 92648], edge_type=[92648], edge_score=[92648]),
  -1),
 (Data(x=[3797, 18], edge_index=[2, 22650], edge_type=[22650], edge_score=[22650]),
  Data(x=[8646, 18], edge_index=[2, 117772], edge_type=[117772], edge_score=[117772]),
  1),
 (Data(x=[3583, 18], edge_index=[2, 59406], edge_type=[59406], edge_score=[59406]),
  Data(x=[8646, 18], edge_index=[2,

In [60]:
len(data)

2000

In [61]:
model = MGCN(data[0][0].x.shape[1], 64, 16)

In [62]:
model.load_state_dict(torch.load(args.MODEL_SAVE_PATH_6_2, map_location='cpu')) 

<All keys matched successfully>

In [63]:
model.eval()

MGCN(
  (pre_mp): Sequential(
    (0): Linear(in_features=18, out_features=64, bias=True)
    (1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
  )
  (batch_norm): BatchNorm1d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (layers): ModuleList(
    (0-7): 8 x MGCNLayer(
      (convs): ModuleList(
        (0-4): 5 x GCNConv(64, 64)
      )
      (skip): Linear(in_features=64, out_features=64, bias=True)
      (norm): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (act): ReLU()
    )
  )
  (post_mp): Sequential(
    (0): Linear(in_features=64, out_features=64, bias=True)
    (1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): Dropout(p=0.3, inplace=False)
    (4): Linear(in_features=64, out_features=32, bias=True)
    (5): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): ReLU()
  

In [64]:
model.eval()
emb_list = []
lbl_list = []


for data_A, data_B, lbl in data:
    eA = model(data_A) 
    eB = model(data_B)  
    
    feat = torch.cat([eA, eB], dim=1)       
    
    emb_list.append(feat.cpu().detach().numpy().ravel())  
    lbl_list.append(1 if lbl==1 else 0)

In [65]:
X = np.stack(emb_list, axis=0)
y = np.array(lbl_list)

In [66]:
X.shape

(2000, 32)

In [67]:
y.shape

(2000,)

In [68]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

In [69]:

clf = LogisticRegression(max_iter=1000).fit(X_train, y_train)

In [70]:
y_pred = clf.predict(X_test)
acc = accuracy_score(y_test, y_pred)
print(f"Accuracy = {acc*100:.2f}%")

Accuracy = 79.50%


In [4]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
with open('/home/rida/GNN/Search_phase/Motifs_data/Motifs_k=5_N=100_s=1_2000_pairs_5_k-hop_16.json', 'r', encoding='utf-8') as f:
    data = json.load(f)


scores = [block[0]['score'] for block in data if isinstance(block, list) and block]
min_score = min(scores)
max_score = max(scores)

print(f"Score minimum : {min_score}")
print(f"Score maximum : {max_score}")

Score minimum : 0.0
Score maximum : 6.268363358685747e-05


In [ ]:
json_files = glob.glob(os.path.join('/home/rida/GNN/Search_phase/Motifs_data', '*.json'))

for path in json_files:
    with open(path, 'r', encoding='utf-8') as f:
        data = json.load(f)
    counter_size = Counter()
    for block in data:
        if isinstance(block, list) and block:
            size = block[0].get('size')
            if size is not None:
                counter_size[size] += 1

    print(f"\n=== Distribution des sizes dans « {os.path.basename(path)} » ===")
    for size, count in sorted(counter_size.items()):
        print(f"• motif avec size = {size} : {count} occurrence(s)")



=== Distribution des sizes dans « Motifs_k=6_N=1000_s=2_500_pairs_6_k-hop_32.json » ===
• motif avec size = 2 : 1 occurrence(s)
• motif avec size = 3 : 1 occurrence(s)

=== Distribution des sizes dans « Motifs_k=5_N=2500_s=2_2000_pairs_5_k-hop_16.json » ===
• motif avec size = 2 : 8 occurrence(s)

=== Distribution des sizes dans « Motifs_k=6_N=50_s=1_500_pairs_6_k-hop_64.json » ===
• motif avec size = 2 : 50 occurrence(s)
• motif avec size = 3 : 50 occurrence(s)
• motif avec size = 4 : 50 occurrence(s)
• motif avec size = 5 : 50 occurrence(s)
• motif avec size = 6 : 50 occurrence(s)

=== Distribution des sizes dans « Motifs_k=4_N=50_s=1_500_pairs_4_k-hop_32.json » ===
• motif avec size = 2 : 50 occurrence(s)
• motif avec size = 3 : 50 occurrence(s)
• motif avec size = 4 : 50 occurrence(s)

=== Distribution des sizes dans « Motifs_k=5_N=100_s=1_2000_pairs_5_k-hop_16.json » ===
• motif avec size = 2 : 100 occurrence(s)
• motif avec size = 3 : 100 occurrence(s)
• motif avec size = 4 : 10

In [4]:
import sys
sys.path.append('/home/rida/GNN')

import networkx as nx
import Embedding_phase.Generate_data.config as args

In [5]:
G = nx.read_graphml(args.GST_DATA_PATH)